In [9]:
import os
import pandas as pd
import random
import numpy as np
import cv2
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split, DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torchvision.transforms import ToTensor
from PIL import Image
import os
from tqdm import tqdm
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
from torchvision import transforms
from torchinfo import summary
import segmentation_models_pytorch as smp

In [10]:
class DatasetCustom(Dataset):
    def __init__(self, img_dir, label_dir, resize=None, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.resize = resize
        self.transform = transform
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        return len(self.images)
    
    def read_mask(self, mask_path):
        image = cv2.imread(mask_path)
        image = cv2.resize(image, self.resize)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        lower_red1 = np.array([0, 100, 20])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160,100,20])
        upper_red2 = np.array([179,255,255])
        
        lower_mask_red = cv2.inRange(image, lower_red1, upper_red1)
        upper_mask_red = cv2.inRange(image, lower_red2, upper_red2)
        
        red_mask = lower_mask_red + upper_mask_red
        red_mask[red_mask != 0] = 1

        green_mask = cv2.inRange(image, (36, 25, 25), (70, 255, 255))
        green_mask[green_mask != 0] = 2

        full_mask = cv2.bitwise_or(red_mask, green_mask)
        full_mask = np.expand_dims(full_mask, axis=-1) 
        full_mask = full_mask.astype(np.uint8)
        
        return full_mask

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx])
        image = cv2.imread(img_path)  #  BGR
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert to RGB
        label = self.read_mask(label_path)  
        image = cv2.resize(image, self.resize)
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [11]:
images_path = "data/train/train/"
image_path = []
TRAIN_DIR = 'data/train/train'
for root, dirs, files in os.walk(TRAIN_DIR):
    for file in files:
        path = os.path.join(root,file)
        image_path.append(path)
        
len(image_path)

1000

In [12]:
mask_path = []
TRAIN_MASK_DIR = 'data/train_gt/train_gt'
for root, dirs, files in os.walk(TRAIN_MASK_DIR):
    for file in files:
        path = os.path.join(root,file)
        mask_path.append(path)
        
len(mask_path)

1000

In [13]:
dataset = DatasetCustom(img_dir= TRAIN_DIR,
                             label_dir= TRAIN_MASK_DIR,
                             resize= (256,256),
                             transform = None)

In [ ]:
import segmentation_models_pytorch as smp

model = smp.UnetPlusPlus(
    encoder_name="resnet34",        
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)

In [14]:
batch_size = 8
learning_rate = 0.0001
color_dict= {0: (0, 0, 0),
             1: (255, 0, 0),
             2: (0, 255, 0)}
best = 10000
origin_epochs = 50
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
images_data = []
labels_data = []
for x,y in dataset:
    images_data.append(x)
    labels_data.append(y)

In [16]:
class UNetDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __getitem__(self, index):
        image = self.data[index]
        label = self.targets[index]
        assert image.shape[:2] == label.shape[:2]
        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image'].float()
            label = transformed['mask'].float()
            label = label.permute(2, 0, 1)
        return image, label
    
    def __len__(self):
        return len(self.data)

In [23]:
transformation = A.Compose([
    A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
    A.HorizontalFlip(p=0.3),
    A.VerticalFlip(p=0.3),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [18]:
train_size = 0.9
val_size = 0.1
torch.manual_seed(123)
train_set, valid_set = random_split(UNetDataset(images_data, labels_data, transformation),
                                    [int(train_size * len(images_data)) , 
                                     int(val_size * len(images_data))])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True)

In [20]:
def mask_to_rgb(mask, color_dict):
    output = np.zeros((mask.shape[0], mask.shape[1], 3))

    for k in color_dict.keys():
        output[mask==k] = color_dict[k]

    return np.uint8(output)    

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [16]:
model.to(device)

epoch_bar = tqdm(total=origin_epochs, desc='Total Progress')

for epoch in range(origin_epochs):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        labels = labels.squeeze(dim=1).long()
        outputs = model(images)
    
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            labels = labels.squeeze(dim=1).long()
            
            outputs = model(images)

            val_loss += criterion(outputs.float(),labels.long()).item()

    print(f"Epoch [{epoch+1}/{origin_epochs}], Loss: {val_loss/len(val_loader):.10f}")
    if val_loss < best:
        best = val_loss
        checkpoint = { 
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': val_loss,
        }
        save_path = f'model.pth'
        torch.save(checkpoint, save_path)
        
    epoch_bar.update(1)
    
epoch_bar.close()

Total Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch [1/50], Loss: 0.4592715752


Total Progress:   2%|▏         | 1/50 [01:36<1:18:37, 96.28s/it]

Epoch [2/50], Loss: 0.2392044294


Total Progress:   4%|▍         | 2/50 [02:48<1:05:46, 82.22s/it]

...

Epoch [48/50], Loss: 0.0508034969


Total Progress:  98%|█████████▊| 49/50 [1:06:57<01:16, 76.07s/it]

Epoch [49/50], Loss: 0.0514693623


Total Progress: 100%|██████████| 50/50 [1:08:14<00:00, 81.89s/it]

Epoch [50/50], Loss: 0.0473644343


In [22]:
num_epochs = 10  
save_path = 'model.pth'
checkpoint = torch.load(save_path)
model.to(device)
model.load_state_dict(checkpoint['model'])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer.load_state_dict(checkpoint['optimizer'])
start_epoch = checkpoint['epoch'] + 1
best = checkpoint['loss']
criterion = nn.CrossEntropyLoss()
epoch_bar = tqdm(total=num_epochs, desc='Total Progress')
for epoch in range(start_epoch, start_epoch + num_epochs):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        labels = labels.squeeze(dim=1).long()
        outputs = model(images)
    
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    
    
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            labels = labels.squeeze(dim=1).long()
            
            outputs = model(images)
            val_loss += criterion(outputs.float(), labels.long()).item()


    print(f"Epoch [{epoch+1}/{start_epoch + num_epochs}], Loss: {val_loss/len(val_loader):.10f}")


    if val_loss < best:
        best = val_loss
        checkpoint = { 
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': val_loss,
        }
        torch.save(checkpoint, save_path)

    epoch_bar.update(1)
    
epoch_bar.close()


Total Progress:  10%|█         | 1/10 [01:16<11:31, 76.82s/it]

Epoch [24/33], Loss: 0.0399129758


Total Progress:  20%|██        | 2/10 [02:37<10:34, 79.34s/it]

Epoch [25/33], Loss: 0.0449616373


Total Progress:  30%|███       | 3/10 [04:01<09:28, 81.18s/it]

...

Epoch [31/33], Loss: 0.0428382906


Total Progress:  90%|█████████ | 9/10 [12:41<01:25, 85.09s/it]

Epoch [32/33], Loss: 0.0417774905


Total Progress: 100%|██████████| 10/10 [14:07<00:00, 84.75s/it]

Epoch [33/33], Loss: 0.0387759328


In [24]:
checkpoint = torch.load('model.pth')
model.load_state_dict(checkpoint['model'])
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model.to(device)

UnetPlusPlus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tru

In [25]:
model.eval()
for i in os.listdir("data/test/test"):
    img_path = os.path.join("data/test/test", i)
    ori_img = cv2.imread(img_path)
    ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    ori_w = ori_img.shape[0]
    ori_h = ori_img.shape[1]
    img = cv2.resize(ori_img, (256, 256))
    transformed = val_transformation(image=img)
    input_img = transformed["image"]
    input_img = input_img.unsqueeze(0).to(device)
    with torch.no_grad():
        output_mask = model.forward(input_img).squeeze(0).cpu().numpy().transpose(1,2,0)
    mask = cv2.resize(output_mask, (ori_h, ori_w))
    mask = np.argmax(mask, axis=2)
    mask_rgb = mask_to_rgb(mask, color_dict)
    mask_rgb = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite("prediction/{}".format(i), mask_rgb) 

In [27]:
import numpy as np
import pandas as pd
import cv2
import os

def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 225] = 255
    pixels[pixels <= 225] = 0
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = 'BKAI-IGH/prediction'
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

df.to_csv(r'submission2.csv', index=False)

prediction\019410b1fcf0625f608b4ce97629ab55.jpeg
prediction\02fa602bb3c7abacdbd7e6afd56ea7bc.jpeg
prediction\0398846f67b5df7cdf3f33c3ca4d5060.jpeg
prediction\05734fbeedd0f9da760db74a29abdb04.jpeg
prediction\05b78a91391adc0bb223c4eaf3372eae.jpeg
prediction\0619ebebe9e9c9d00a4262b4fe4a5a95.jpeg
prediction\0626ab4ec3d46e602b296cc5cfd263f1.jpeg
prediction\0a0317371a966bf4b3466463a3c64db1.jpeg
prediction\0a5f3601ad4f13ccf1f4b331a412fc44.jpeg
prediction\0af3feff05dec1eb3a70b145a7d8d3b6.jpeg
prediction\0fca6a4248a41e8db8b4ed633b456aaa.jpeg
prediction\1002ec4a1fe748f3085f1ce88cbdf366.jpeg
prediction\1209db6dcdda5cc8a788edaeb6aa460a.jpeg
prediction\13dd311a65d2b46d0a6085835c525af6.jpeg
prediction\1531871f2fd85a04faeeb2b535797395.jpeg
prediction\15fc656702fa602bb3c7abacdbd7e6af.jpeg
prediction\1ad4f13ccf1f4b331a412fc44655fb51.jpeg
prediction\1b62f15ec83b97bb11e8e0c4416c1931.jpeg
prediction\1c0e9082ea2c193ac8d551c149b60f29.jpeg
prediction\1db239dda50f954ba59c7de13a35276a.jpeg
prediction\26679bff5